In [1]:
%matplotlib inline
%pip install -q mediapy
import mediapy as media
media.set_ffmpeg('/opt/bwhpc/common/vis/ffmpeg/5.1.3/bin/ffmpeg')

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import numpy as np
import torch

import yaml
from stable_baselines3 import PPO
from sb3_contrib import RecurrentPPO

from ppo_airhockey_benchmark.constants import best_model_file_name, vecnormalize_file_name
from ppo_airhockey_benchmark.util import make_environments

/home/kit/stud/uqtlv/.conda/envs/airhockey2/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [3]:
def prepare(local_testing):
    global q
    q = 0
    env = local_testing["env"]
    env_args = local_testing["env_args"]
    path = pathlib.Path('./' + local_testing["path"])
    eval_env = make_environments(env, 0, env_args, only_eval=True, load=True, load_dir=path / vecnormalize_file_name)
    model_load_dir = path / best_model_file_name
    if local_testing["model_type"] == 'recurrent':
        model = RecurrentPPO.load(model_load_dir)
    else:
        model = PPO.load(model_load_dir)
    
    return eval_env, model

def eval(eval_env, model, local_testing):
    global q
    obs = eval_env.reset()
    cum_reward = 0
    images = []
    if local_testing["model_type"] == 'recurrent':
        lstm_states = None
        num_envs = 1
        episode_starts = np.ones((num_envs,), dtype=bool)
        with torch.no_grad():
            while True:
                if local_testing["model_type"] == 'recurrent':
                    action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True)
                else:
                    action, state = model.predict(obs, deterministic=True)
                obs, reward, done, info = eval_env.step(action)
                cum_reward += reward
                images.append(eval_env.render(mode='rgb_array'))
                if done:
                    print("Run {}".format(q))
                    print("Reward until last:{}".format(cum_reward - reward))
                    print("Full reward:{}".format(cum_reward))
                    q+=1
                    return images
        
def show_eval(*args, **kwargs):
    frames = eval(*args, **kwargs)
    media.show_video(frames, fps=50, width=500)
    return frames

In [4]:
def run_n(local_testing, n=50):
    display(local_testing)
    eval_env, model = prepare(local_testing)
    for i in range(n):
        show_eval(eval_env, model, local_testing)

In [ ]:
local_testing = {
    'model_type': 'recurrent',
    'path': 'logs/RnnPPO/7dof_defend/test-recurrent-3',
    'env': '7dof-defend',
    'env_args': {'noise': True, 'horizon': 300, 'last_joint': True}
}

run_n(local_testing, n=15)

In [5]:
local_testing = {
    'model_type': 'normal',
    'path': 'logs/RnnPPO/7dof_defend/test-ppo-3',
    'env': '7dof-defend',
    'env_args': {'noise': True, 'horizon': 300, 'last_joint': True}
}

run_n(local_testing, n=15)

{'model_type': 'normal',
 'path': 'logs/RnnPPO/7dof_defend/test-ppo-3',
 'env': '7dof-defend',
 'env_args': {'noise': True, 'horizon': 300, 'last_joint': True}}

TypeError: 'NoneType' object is not iterable